Embedding Book Metadata with all-MiniLM-L6-v2

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
os.chdir("..")  # Go up to project root

In [4]:
df_books = pd.read_csv("C:/Users/jvlas/source/repos/TrioLearn/data/interim/books_metadata.csv")

In [5]:
# Fill missing fields (avoid errors during string joining)
df_books.fillna("", inplace=True)

C:\Users\jvlas\AppData\Local\Temp\ipykernel_54612\1538531904.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_books.fillna("", inplace=True)


In [6]:
# Combine text fields into one string for embedding
df_books["text_for_embedding"] = (
    df_books["title"] + " " +
    df_books["description"] + " " +
    df_books["categories"]
)

In [7]:
# Load the pre-trained Sentence-BERT model

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
print(" Model loaded")



c:\Users\jvlas\source\repos\TrioLearn\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Model loaded


In [8]:
#  Compute embeddings (batch process for performance)
embeddings = model.encode(df_books["text_for_embedding"].tolist(), show_progress_bar=True)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.34s/it]


In [9]:
# Store as separate columns
embedding_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(embeddings.shape[1])])


In [10]:
# Concatenate with original metadata
df_books_embedded = pd.concat([df_books, embedding_df], axis=1)

In [11]:
processed_dir = os.path.join("data", "processed")
os.makedirs(processed_dir, exist_ok=True)

save_path = os.path.join(processed_dir, "books_with_embeddings.csv")
df_books_embedded.to_csv(save_path, index=False)

print(f"Saved {len(df_books_embedded)} books with embeddings to:", save_path)

Saved 20 books with embeddings to: data\processed\books_with_embeddings.csv
